In [42]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
import warnings
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

warnings.filterwarnings("ignore")

In [96]:
#connecting the webdriver
driver = webdriver.Chrome(r'C:\Users\AKHIL\Downloads\chromedriver_win32\chromedriver.exe')

In [97]:
#maximizing the window just in case for any data not to be missed
driver.maximize_window()

The reviews and ratings will be scraped for the following products:-

1) Phones, 
2) Laptops, 
3) Headphones, 
4) Smart watches, 
5) Professional cameras, 
6) Printers, 
7) Monitors, 
8) Home theatre, 
9) Router

The data will be scraped from amazon, flipkart, snapdeal untill the target of data points is acheived

In [98]:
# Loading amazon.in
url = "https://www.amazon.in/"
driver.get(url)

Need to scrape atleast 20000 rows with equal amount of 1star , 2star, 3star, 4star and 5star values in order to avoid data imbalances in ratings column while building a model.

approximately 4000 rows for each star to perfectly balance

In [122]:
#List of products for which reviews has to be collected
products = ["Smart Phone","Laptop", "Headphones", "Smart watch",
            "Professional camera", "Printer", "Monitor", "Home theater", "Router"]

#list of data values to be collected for
columns = ["Product type", "Title", "Review", "Rating"]

#creating an empty dataframe to which the data scraped will be added
df = pd.DataFrame({} , columns = columns)

There might be some exceptions raised beacuse... 

few reviews have only title and no content , Some products doesnt have even a single review, some products have very less reviews and doesnt need many pages to show all the reviews...

In [123]:
#Code for scraping data from amazon

for p in products: #looping
    
    search_box_amazon = driver.find_element_by_id("twotabsearchtextbox") #search box
    search_btn_amazon = driver.find_element_by_id("nav-search-submit-button") #search button
    
    search_box_amazon.clear() #clearing the text... needed from second loop onwards
    search_box_amazon.send_keys(p) #input the keyword to search
    search_btn_amazon.click() #clicking on the search button
    
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #scrolling to the bottom just to load the entire page
    
    
    for number in range (0,2): # loop for scraping only from 3 pages of search list generated
        search_list = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//h2//a//span") #products in search list

        for i in search_list:
            product = i.text
            try:
                i.click() #clicking on the product to open its page
            except:
                continue
            
            driver.switch_to.window(driver.window_handles[-1]) #switching to the product page
            time.sleep(1)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            try:
                stars = driver.find_element_by_xpath("//table[@id = 'histogramTable']//tr//td[1]//span//a") #ratings element
                stars.click() #proceeding to the ratings page
            except:
                print("No reviews for the product " , product) #if no reviews, helps us to know
    
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(2)
        
            stars_select = driver.find_elements_by_xpath("//table[@id = 'histogramTable']//tr//td[1]//span//a") #list of rating stars
        
            for s in stars_select: #each loop - each star
                s.click() #loading selected star reviews
                time.sleep(1)
            
            
                for num in range(0,2): #reviews in first three pages of selected star
                    
                    reviews_title = driver.find_elements_by_xpath("//div[@data-hook = 'review']//a[@data-hook = 'review-title']") #just knowing the no.of reviews
        
                    for j in range(len(reviews_title)): #helps to scrape individual row and adding to dataframe
                        
                        row = [] #empty row
                        
                        try: #to handle the missing review content
                            
                            #specific title and content for same element
                            reviews_title = driver.find_element_by_xpath("//div[@data-hook = 'review'][{}]//a[@data-hook = 'review-title']".format(j+1))
                            reviews_content =  driver.find_element_by_xpath("//div[@data-hook = 'review'][{}]//span[@data-hook = 'review-body']//span".format(j+1))
                        
                            #creating the data row
                            row.append(p)
                            row.append(reviews_title.text)
                            row.append(reviews_content.text)
                            row.append(s.text)
                        except:
                            continue #skips the next part in loop if anything is missing in the above data row
                            
                        df1 = pd.DataFrame([row] , columns=columns)
                        df = pd.concat([df,df1],ignore_index=True)
                
                    try: #to handle no next page error
                        next_btn  = driver.find_element_by_xpath("//div[@role='navigation']//li[@class='a-last']//a")
                        next_btn.click()
                        time.sleep(2)
                    except:
                        break #breaks the current loop if there are no more reviews.
        
            driver.close() #closing the reviews page
            driver.switch_to_window(driver.window_handles[0]) #switching to the main page (search list page)
            
            
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        
        #next page of search_list
        next_btn  = driver.find_element_by_xpath("//div[@role='navigation']//ul[@class='a-pagination']//li[@class='a-last']//a")
        next_btn.click()
        time.sleep(1)

No reviews for the product  I KALL K320 Smartphone (6.53 HD+ Display, 4GB Ram, 64GB Storage, Dual Sim, 4G Volte) (Blue)
No reviews for the product  I KALL K5 Smartphone (5.5 Inch, 2GB Ram, 16GB Storage, Dual Sim, 4G Volte)
No reviews for the product  I KALL K200 Smartphone (5.5 Inch, 2GB, 16GB) (Gold)
No reviews for the product  Dell Inspiron 3505 15.6" HD Display Laptop (Athlon Silver 3050U / 4GB / 256GB SSD / Integrated Graphics / Win10 + MSO) D560483WIN9BE
No reviews for the product  Lenovo IdeaPad Slim 3i (82H801CSIN) 15.6" FHD Laptop (Core i5-1135G7 / 8GB /256GB SSD /Iris Xe Graphics /Win 10 /Office'19) Arctic Grey
No reviews for the product  ASUS VivoBook 14 (2020), 14-inch (35.56 cms) Full HD, AMD Ryzen 3 3250U, Thin and Light Laptop (4GB RAM/1TB HDD/Integrated Graphics/Windows 10/Slate Grey/1.6 kg), M415DA-EB301T
No reviews for the product  (Renewed) Dell Latitude Laptop E5470 Intel Core i5 6th Gen. - 6200u Processor, 8 GB Ram & 512 GB SSD, 14.1 Inches HD Screen Notebook Comput

In [124]:
# just to check the number of rows scraped.
df.tail()

,Product type,Title,Review,Rating
23039,Router,Below Average. Doesn’t meet expectations.,Below Average. Every year antivirus Netgear Ar...,1 star
23040,Router,This brand is no more what it once was!,Don't buy this. There is constant drop in conn...,1 star
23041,Router,Waste of money and no support from Netgear,"Don't buy this product, they said it increases...",1 star
23042,Router,Waste of Money,Network not reachable if door closed. the dist...,1 star
23043,Router,The seller sent a used piece,The seller sent a used piece. No value for mon...,1 star


In [127]:
#checking if number of rows for each star
df.Rating.value_counts()

5 star    5516
1 star    4881
4 star    4784
3 star    4163
2 star    3700
Name: Rating, dtype: int64

It looks like the rating column is balanced.

In [128]:
# saving the data to csv file
df.to_csv("Ratings_data.csv")

In [129]:
df.isnull().sum()

Product type    0
Title           0
Review          0
Rating          0
dtype: int64